In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt

def get_cap_series(C1, C2):
     return C1*C2 / (C1 + C2)
 
def get_cap_parallel(C1, C2):
    return C1 + C2

def get_junction_chain_capacitance(Cj1, Cj2, Cj_gnd, n, C_load):
    """
    Calculate the capacitance of a junction array to GND.
    Note the connecting leads at the beginning and end are not included. 
            n=1           n=2       ...  n=N
            Cj1     Cj2   Cj1     Cj2    Cj1
         ---||-----||-----||-----||  ... ||--- 
                |      |                    |
                =      = Cj_GND             =   C_load(only last one)
                |      |                    |
         ------------------------------------- GND
    
    Parameters
    ----------
    Cj1 : float
        Junction capacitance. 50~100 fF/um^2
    Cj2 : float
        parasitic junction capacitance
    Cj_gnd : float
        capacitance between a parasitic junction and GND
    n : int
        number of junctions.
    C_load : float
        capacitance of load to GND
    Returns
    -------
    float
        capacitance

    """
         
    if not isinstance(n, int):
        raise TypeError('n must be integer!!!')
    if n==1:
        return get_cap_series(Cj1, C_load)
    else:
        C_n_minus_1 = get_junction_chain_capacitance(Cj1, Cj2, Cj_gnd, n-1, C_load)
        C_n = get_cap_series(Cj1, get_cap_parallel(
            Cj_gnd, get_cap_series(Cj2, get_cap_parallel(
                Cj_gnd,C_n_minus_1))))
        return C_n
    
def get_single_JJ_capacitance(Cj1, Cj2, Cj_gnd, C_load):
    """
    Calculate the capacitance of a single JJ + two leads to GND for three cases.
    Most likely, the third one would be the right one.
                    Cj1          Cj2               
         ----------||------------||--------------
              |           |           |         |
              = 2*Cj_gnd  = Cj_gnd    = Cj_gnd  = C_load
              |           |           |         |   
         ----------------------------------------------- GND
    
                    Cj2          Cj1   
         ----------||------------||---------------
              |           |           |          |        
              = Cj_gnd    = Cj_gnd    = 2*Cj_gnd = C_load
              |           |           |          |          
         ----------------------------------------------- GND
    
                    Cj1               
         ----------||------------------
              |           |           |
              = 2*Cj_gnd  = 2*Cj_gnd  = C_load
              |           |           |          
         ----------------------------------------------- GND
    
    Parameters
    ----------
    Cj1 : float
        Junction capacitance. 50~100 fF/um^2
    Cj2 : float
        parasitic junction capacitance
    Cj_gnd : float
        capacitance between a parasitic junction and GND
    C_load : float
        capacitance of load to GND
        
    Returns
    -------
    float
        capacitance

    """
    a0 = get_cap_parallel(Cj_gnd, C_load)        
    a1 = get_cap_series(Cj2, a0)
    a2 = get_cap_parallel(Cj_gnd, a1)
    a3 = get_cap_series(Cj1, a2)
    a4 = get_cap_parallel(2*Cj_gnd, a3)
        
    b0 = get_cap_parallel(2*Cj_gnd, C_load)        
    b1 = get_cap_series(Cj1, b0)
    b2 = get_cap_parallel(Cj_gnd, b1)
    b3 = get_cap_series(Cj2, b2)
    b4 = get_cap_parallel(Cj_gnd, b3)
        
    c0 = get_cap_parallel(2*Cj_gnd, C_load)        
    c1 = get_cap_series(Cj1, c0)
    c2 = get_cap_parallel(2*Cj_gnd, c1)
  
    
    return a4, b4, c2

Using matplotlib backend: TkAgg


# Junction array behavior

In [87]:
Cj1 = 10  # fF
Cj2_list = Cj1 * np.array([2,5,10,20])
Cj_gnd_list = np.linspace(0.01, 100, 51)
C_load = 1
n = 10

fig, ax = plt.subplots(1,1, figsize=(8,6))
for Cj2 in Cj2_list:
    island_cap_list = [get_junction_chain_capacitance(Cj1, Cj2, Cj_gnd, n, C_load) for Cj_gnd in Cj_gnd_list]
    ax.plot(Cj_gnd_list, island_cap_list, label=f'Cj2={Cj2} fF')
ax.set_xlabel('Cj_gnd (fF)',fontsize=18)
ax.set_ylabel(r'Total capacitance (fF)',fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Cj1 = {Cj1:.1f} fF, # of junction = {n}, C_load={C_load:.1f} fF', fontsize=18)
ax.grid()
ax.legend(fontsize=18, loc=4)

Cj1 = 10  # fF
Cj2 = Cj1 * 10
Cj_gnd_list = [0.05, 0.1, 0.5, 1, 5 ] #np.linspace(0.01, 10, 11)
n_list  = range(1, 20)

fig, ax = plt.subplots(1,1, figsize=(8,6))
for Cj_gnd in Cj_gnd_list:
    island_cap_list = [get_junction_chain_capacitance(Cj1, Cj2, Cj_gnd, n, C_load) for n in n_list]
    ax.plot(n_list, island_cap_list, '-o', label=f'Cj_gnd={Cj_gnd} fF')
ax.set_xlabel('# of junctions',fontsize=18)
ax.set_ylabel(r'Total capacitance (fF)',fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_xlim([0,20.5])
ax.set_title(f'Cj1 = {Cj1:.1f} fF, Cj2 = {Cj2:.1f} fF, C_load={C_load:.1f} fF', fontsize=18)
ax.grid()
ax.legend(fontsize=18, loc=1)


In [28]:
Cj1  = 10
Cj2 = 100
Cj_gnd = 0.35
C_load = 50
n = 10
get_junction_chain_capacitance(Cj1, Cj2, Cj_gnd,n, C_load)

2.242128925892423

# Single JJ behavior

In [81]:
Cj1  = 1.4
Cj2 = 14
Cj_gnd_list = np.array([0.1, 0.5, 1,2])
C_load_list = np.linspace(0.01,5, 101)
get_single_JJ_capacitance(Cj1, Cj2, Cj_gnd, C_load)

fig, ax = plt.subplots(1,1, figsize=(8,6))
for Cj_gnd in Cj_gnd_list:
    cap = np.array([get_single_JJ_capacitance(Cj1, Cj2, Cj_gnd, C_load)[2] for C_load in C_load_list])
    ax.plot(C_load_list, cap, label=f'Cj_gnd={Cj_gnd:.1f}')
ax.set_xlabel('C_load (fF)',fontsize=18)
ax.set_ylabel(r'Total capacitance (fF)',fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Cj1 = {Cj1:.1f} fF', fontsize=18)
ax.grid()
ax.legend(fontsize=18, loc=1)

# Single JJ + junction array arm capacitance

In [6]:
# small JJ
Cj1_s = 1.7
Cj2_s = 2*Cj1_s
Cj_gnd_s = 0.05

# junction array
Cj1_a = 10
Cj2_a = 2*Cj1_a
Cj_gnd_a = 0.05
n = 10

C_pad = 0.5 # capacitance of pad to gnd that connects small JJ and junction arraay
C_load = 50  # capacitance of whatever connected at the end of either small jj or array.

# define sweep
# sweep_list = np.linspace(0.1, 100, 201)
sweep_list = np.linspace(0.01, 1, 101)
# sweep_str = 'C_load'
# sweep_str = 'Cj_gnd_a, Cj_gnd_s'
sweep_str = 'Cj_gnd_a'
# C_load = sweep_list
Cj_gnd_a = sweep_list
# Cj_gnd_s = sweep_list

JJ_array = get_junction_chain_capacitance(Cj1_a, Cj2_a, Cj_gnd_a, n, C_load)
single_and_array = get_single_JJ_capacitance(Cj1_s, Cj2_s, Cj_gnd_s
                                             , get_cap_parallel(C_pad, JJ_array)
                                            )[2]

single_JJ = get_single_JJ_capacitance(Cj1_s, Cj2_s, Cj_gnd_s, C_load)[2]
array_and_single = get_junction_chain_capacitance(Cj1_a, Cj2_a, Cj_gnd_a, n
                                                  , get_cap_parallel(C_pad, single_JJ)
                                                 )

# plot
fig, ax = plt.subplots(1,1, figsize=(9,7))
ax.plot(sweep_list, single_and_array, label=f'single + array')
ax.plot(sweep_list, array_and_single, label=f'array + single')
ax.set_xlabel(sweep_str + ' (fF)',fontsize=18)
ax.set_ylabel(r'Total capacitance (fF)',fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'small JJ, Cj1 = {Cj1_s:.1f} fF, Cj2 = {Cj1_s:.1f} fF\n \
            array, Cj1 = {Cj1_a:.1f} fF, Cj2 = {Cj2_a:.1f} fF, n = {n}\n \
            C_pad = {C_pad:.1f}, C_load = {C_load:.1f}', fontsize=18)
ax.grid()
ax.legend(fontsize=18, loc=1)

In [47]:
# small JJ
Cj1_s = 1.7
Cj2_s = 20 * Cj1_s
Cj_gnd_s = 0.15

# junction array
Cj1_a = 10
Cj2_a = 2 *  Cj2_a
Cj_gnd_a = 0.15
n = 10

C_pad = 0
C_load = 50  # capacitance of whatever connected at the end.

C_center_island = 0.5

JJ_array = get_junction_chain_capacitance(Cj1_a,Cj2_a,Cj_gnd_a, n, C_load)
single_and_array = get_single_JJ_capacitance(Cj1_s, Cj2_s, Cj_gnd_s
                                             , get_cap_parallel(C_pad, JJ_array)
                                            )[2]

single_JJ = get_single_JJ_capacitance(Cj1_s, Cj2_s, Cj_gnd_s,C_load)[2]
array_and_single = get_junction_chain_capacitance(Cj1_a, Cj2_a, Cj_gnd_a, n
                                                  , get_cap_parallel(C_pad, single_JJ)
                                                 )

print(f'single + array = {single_and_array:.3f} fF')
# print(f'center island =  {C_center_island:.3f} fF')
# print(f'total = {single_and_array+C_center_island:.3f} fF\n')
      
# print(f'array + single = {array_and_single:.3f} fF')
# print(f'center island =  {C_center_island:.3f} fF')
# print(f'total = {array_and_single+C_center_island:.3f} fF')

single + array = 1.218 fF


# Unit conversion

In [12]:
from scipy.constants import h, e, k, hbar, epsilon_0

def K_to_C(temperature):
    """ Kelvin to capacitance assuming 2e charging energy"""
    energy = k * temperature
    
    return (2*e)**2/2/energy

def C_to_K(cap):
    """ Capacitance to Kelvin assuming 2e charging energy"""
    charging_energy = (2*e)**2 / 2/cap
    
    return charging_energy / k


In [48]:
C_to_K((1.6)*1e-15)

2.3240617425915424

In [44]:
1.6+1.218*4

6.4719999999999995